# Data Staging
### Extract, Transform, Load (ETL)

**Create Data Frame for Facility Locations in Ontario**


In [373]:
import pandas as pd

Extract Facility Location

In [374]:
# read file containing information about recreation centre locations and types
# facility location dataframe
df = pd.read_csv('./ODRSF_v1.0.csv', encoding='cp1252')

C:\Users\Arthur\AppData\Local\Temp\ipykernel_13944\371614087.py:3: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./ODRSF_v1.0.csv', encoding='cp1252')


Transform Facility Dataframe

In [375]:
# Remove unneeded columns
df = df.drop(columns = ['Index', 'Facility_Name', 'Source_Facility_Type', 'Provider', 'City', 'Source_Format_Address', 'CSD_UID', 'PR_UID', 'Latitude', 'Longitude'])

# Remove unit since almost all values are null
df = df.drop(columns = ['Unit'])

In [376]:
#Replace empty values of street_no with -1
df.loc[df['Street_No'] == '..', 'Street_No'] = '-1'

# Replace rest of street strings with an empty value
df.loc[df['Street_Name'] == '..', 'Street_Name'] = ''
df.loc[df['Street_Type'] == '..', 'Street_Type'] = ''
df.loc[df['Street_Direction'] == '..', 'Street_Direction'] = ''
df.loc[df['Street_Direction'] == '..', 'Street_Direction'] = ''
df.loc[df['Postal_Code'] == '..', 'Postal_Code'] = ''

In [377]:
#convert street_no to integer
df['Street_No'] = df['Street_No'].astype(int)

In [378]:
# Captialize all province/territory column
df['Prov_Terr'] = df['Prov_Terr'].str.upper()

In [379]:
# remove all non ontario provinces
df = df[df['Prov_Terr'] == 'ON']

In [380]:
# replace sudbury areas with just sudbury

# Remove unnecessary information after comma in geo_name so processing can be easier
df['CSD_Name'] = df['CSD_Name'].str.split(',').str[0]

In [381]:
#unique_cities = df['City'].unique()
unique_cities = df['CSD_Name'].unique()

In [382]:
column_rename_dict = {
    'CSD_Name': 'City',
    'ODRSF_facility_type': 'FacilityType',
}

df = df.rename(columns=column_rename_dict)

Generating Surrogate Key for Location Dataframe

In [383]:
df['Facility ID'] = range(1,len(df)+1)

df = df.reindex(columns=['Facility ID'] + list([c for c in df.columns if c!= 'Facility ID']))

**Create Dataframe for City Demographics in Ontario**

In [384]:
demographic_df = pd.read_csv('./Ontario_Demographics.csv', encoding='cp1252')

In [385]:
demographic_df = demographic_df.drop(columns = ['CENSUS_YEAR', 'DGUID', 'ALT_GEO_CODE', 'GEO_LEVEL', 'TNR_SF', 'TNR_LF', 'DATA_QUALITY_FLAG','SYMBOL', 'CHARACTERISTIC_ID', 'CHARACTERISTIC_NOTE', 'C2_COUNT_MEN+', 'C3_COUNT_WOMEN+', 'C10_RATE_TOTAL', 'C11_RATE_MEN+', 'C12_RATE_WOMEN+', 'SYMBOL.1','SYMBOL.2','SYMBOL.3','SYMBOL.4','SYMBOL.5'])

In [386]:
# Remove unnecessary information after comma in geo_name so processing can be easier
demographic_df['GEO_NAME'] = demographic_df['GEO_NAME'].str.split(',').str[0]

# only keep rows where the city is both in the demographic dataframe and the facility location dataframe
demographic_df = demographic_df[demographic_df['GEO_NAME'].isin(unique_cities)]

In [387]:
#get starting and ending index of rows that contain age information about the city
age_indexes = demographic_df[demographic_df['CHARACTERISTIC_NAME'].str.contains('Total - Age groups of the population - 100% data|100 years and over')].index

#get starting and ending index of rows that contain ethnicity information about the city
ethnicity_indexes = demographic_df[demographic_df['CHARACTERISTIC_NAME'].str.contains('Total visible minority population|Total - Ethnic or cultural origin for the population in private households - 25% sample data')].index

In [388]:
selected_age_dfs = []
selected_ethnicity_dfs = []

# store only data in dataframe from start and end indexes
for i in range(0, len(age_indexes), 2):
    age_df = demographic_df[age_indexes[i] + 1: age_indexes[i + 1] + 1]
    ethnicity_df = demographic_df[ethnicity_indexes[i] + 1: ethnicity_indexes[i + 1]]
    selected_age_dfs.append(age_df)
    selected_ethnicity_dfs.append(ethnicity_df)

#resulting dataframes 
result_age_df = pd.concat(selected_age_dfs, ignore_index=True)
result_ethnicity_df = pd.concat(selected_ethnicity_dfs, ignore_index=True)

# since some 'cities' have multiple occurences(i.e, different regions, we will need to keep each of their unique values.)
# To make the ETL process easier, we want to make it so each city in their respective data occurs only once
prev_city = None
for index, row in result_age_df.iterrows():
    # Check if current city is different from previous city
    if row['GEO_NAME'] == prev_city:
        # Drop the row if the city is the same as the previous one
        result_age_df.at[index, 'GEO_NAME'] = ''
    else:
        # Update previous city
        prev_city = row['GEO_NAME']

# special cases where these CSDs are right after each other so the for loop above does not work
result_age_df.at[4875, 'GEO_NAME'] = 'Nipissing'
result_age_df.at[5200, 'GEO_NAME'] = 'Parry Sound'

prev_city = None
for index, row in result_ethnicity_df.iterrows():
    # Check if current city is different from previous city
    if row['GEO_NAME'] == prev_city:
        # Drop the row if the city is the same as the previous one
        result_ethnicity_df.at[index, 'GEO_NAME'] = ''
    else:
        # Update previous city
        prev_city = row['GEO_NAME']    

# special cases where these CSDs are right after each other so the for loop above does not work
result_ethnicity_df.at[2535, 'GEO_NAME'] = 'Nipissing'
result_ethnicity_df.at[2703, 'GEO_NAME'] = 'Parry Sound'

In [389]:
# set empty values/null to 0
result_age_df.fillna({'C1_COUNT_TOTAL': 0}, inplace=True)
# check for null
result_age_df['C1_COUNT_TOTAL'].isnull().sum()

0

In [390]:
# set empty values/null to 0
result_ethnicity_df.fillna({'C1_COUNT_TOTAL': 0}, inplace=True)
# check for null
result_ethnicity_df['C1_COUNT_TOTAL'].isnull().sum()

0

In [391]:
# get all cities from the dataframe(cities are the same from the age dataframe and tehnicity dataframe)
cities = result_age_df[result_age_df['GEO_NAME'] != '']['GEO_NAME']

Transpose Age Dataframe

In [392]:
# Unique age ranges - will be used as the new column in the new dataframe
ages = result_age_df['CHARACTERISTIC_NAME'].unique()

age_df = pd.DataFrame(columns=ages)

# add new column for city
age_df.insert(loc=0, column='City', value='')

# remove whitespace from columns
age_df = age_df.rename(columns=lambda x: x.strip())

In [393]:
# Populate first column cities with all city variables
age_df['City'] = cities
age_count = result_age_df['C1_COUNT_TOTAL'].tolist()

for i in range(len(cities)):
    if i == 0:
        age_df.iloc[i, 1:] = age_count[:25]
    else:
        age_df.iloc[i, 1:] = age_count[i * 25 : (i + 1) * 25]
        
for column in age_df.columns[1:]:
    age_df[column] = age_df[column].astype(int)

# Sum duplicate rows since we consider regions of a City as the same City, and sort data by City
age_df = age_df.groupby('City').sum().reset_index()

In [394]:
# redundant data that can be dropped
age_df = age_df.drop(columns = ['0 to 14 years', '15 to 64 years','65 years and over'])

# NOTE: We decided to not bucketisize the columns since most of the data errs on one side of the spectrum

In [395]:
# not sure if this is needed since we can easily match through the natural keys
age_df.insert(0, 'Age City ID', '')
age_df['Age City ID'] = ['A' + str(x) for x in range(1, len(age_df) + 1)]

Transpose Ethnicity Dataframe

In [396]:
# Unique ethnicities
ethnicities = result_ethnicity_df['CHARACTERISTIC_NAME'].unique()

ethnicity_df = pd.DataFrame(columns=ethnicities)

ethnicity_df.insert(loc=0, column='City', value='')

# remove whitespace from columns
ethnicity_df = ethnicity_df.rename(columns=lambda x: x.strip())

In [397]:
# Populate first column with all cities
ethnicity_df['City'] = cities
ethnicity_count = result_ethnicity_df['C1_COUNT_TOTAL'].tolist()

# Populate column data
k = 0
for i in range(len(cities)):
    if i == 0:
        ethnicity_df.iloc[i, 1:] = ethnicity_count[:13]
    else:
        ethnicity_df.iloc[i, 1:] = ethnicity_count[i * 13 : (i + 1) * 13]

#change type from float to int
for column in ethnicity_df.columns[1:]:
    ethnicity_df[column] = ethnicity_df[column].astype(int)

# Sum duplicate rows since we consider regions of a City as the same City, and sort data by City
ethnicity_df = ethnicity_df.groupby('City').sum().reset_index()

Generate Surrogate Keys for Ethnicity Dataframe

In [398]:
# not sure if this is needed since we can easily match through the natural keys
ethnicity_df.insert(0, 'Ethnicity City ID', '')
ethnicity_df['Ethnicity City ID'] = ['E' + str(x) for x in range(1, len(ethnicity_df) + 1)]

Generate Facility Type Data Frame

In [399]:
unique_facilities = df['FacilityType'].unique()

facility_type_df = pd.DataFrame()
facility_type_df['Facility Type ID'] = ['FT' + str(x) for x in range(1, len(unique_facilities) + 1)]
facility_type_df['Type'] = unique_facilities

Populate Facility Dataframe with Foreign Keys

In [400]:
df = pd.merge(df, facility_type_df, left_on='FacilityType', right_on='Type', how='left')
df = df.drop(columns = ['FacilityType', 'Type'])

Generate Mappings for Ethnicites and Age Ranges

In [401]:
age_mapping_df = pd.DataFrame()
age_mapping_df['Age_ID'] = range(1,len(age_df.columns[2:])+1)
age_mapping_df['Value'] = age_df.columns[2:]

ethnicity_mapping_df = pd.DataFrame()
ethnicity_mapping_df['Ethnicity_ID'] = range(1,len(ethnicity_df.columns[2:])+1)
ethnicity_mapping_df['Value'] = ethnicity_df.columns[2:]

**Generating Measure/Fact Table**

In [402]:
fact_df = pd.DataFrame()
fact_df['City'] = age_df['City']
fact_df = fact_df.sort_values(by='City').reset_index(drop=True)
fact_df['Age City ID'] = age_df['Age City ID']
fact_df['Ethnicity City ID'] = ethnicity_df['Ethnicity City ID']
fact_df['Facility Count'] = None
fact_df['Highest Population Age ID'] = None
fact_df['Highest Population Ethnicity ID'] = None


for i in range(len(age_df)):
    fact_df.loc[i, 'Highest Population Age ID'] = (age_df.iloc[i, 2:]).idxmax()
    fact_df.loc[i, 'Highest Population Ethnicity ID'] = (ethnicity_df.iloc[i, 2:]).idxmax()
    fact_df.loc[i, 'Facility Count'] = (df['City'] == fact_df.loc[i, 'City']).sum()

In [403]:
# Adding all facility IDs that belong to a city
merged_df = pd.merge(fact_df, df, left_on='City', right_on='City', how='left')

# Group by City and aggregate values of Facility ID into a list
result = merged_df.groupby('City')['Facility ID'].agg(list).reset_index()

# Rename the aggregated column to the desired column name in DataFrame A
result.rename(columns={'Facility ID': 'Facility IDs'}, inplace=True)

# Merge the result back to DataFrame  on X
fact_df = pd.merge(fact_df, result, on='City', how='left')

df = df.drop(columns = ['City'])

Update text value mappings with numeric values for measure data

In [404]:
mapping_dict_age = dict(zip(age_mapping_df['Value'], age_mapping_df['Age_ID']))
fact_df['Highest Population Age ID'] = fact_df['Highest Population Age ID'].map(mapping_dict_age)

mapping_dict_ethnicity = dict(zip(ethnicity_mapping_df['Value'], ethnicity_mapping_df['Ethnicity_ID']))
fact_df['Highest Population Ethnicity ID'] = fact_df['Highest Population Ethnicity ID'].map(mapping_dict_ethnicity)

Load

In [405]:
df.to_csv('facility_df.csv', index=False)
facility_type_df.to_csv('facility_type_df.csv', index=False)
age_df.to_csv('age_df.csv', index=False)
ethnicity_df.to_csv('eth_df.csv', index=False)
fact_df.to_csv('fact_df.csv', index=False)